In [1]:
import os
import numpy as np
import pandas as pd

In [42]:
GUESSES = 20

In [3]:
DATA_PATH = os.path.expanduser('~/Documents/code/trabalho-disciplina/set-expansion-structured-data/data')
RESULT_PATH = os.path.join(DATA_PATH, 'cnae-s-em-output')

In [4]:
dfquery = pd.read_csv(os.path.join(DATA_PATH, 'CNAE-9_query.csv'))

In [53]:
%%time

results = []
for n in [10, 20, 30, 50, 100, 150, 200]:
    precisions = []
    recalls = []
    for query in dfquery.iterrows():
        index = query[0]
        query_items = np.array([int(x) for x in query[1]['query'].split(',')])
        query_targets = np.array([int(x) for x in query[1]['target'].split(',')])

        # these did not work for S-EM
        if index in [160, 206, 239, 305, 324, 408, 429, 438]:
            continue

        with open(os.path.join(RESULT_PATH, str(index) + '.output'), 'r') as output_file:
            scores = output_file.read()

        scores = np.array([float(x) for x in scores.split('\n')[:-1]])
        most_likely = np.argsort(scores)

        # remove the query itself from the results
        valid_guesses = np.where(~np.isin(most_likely, query_items))

        expansion = most_likely[valid_guesses][:n]

        hits = np.sum(np.isin(expansion, query_targets))
        precision = hits / n
        recall = hits / len(query_targets)
        precisions.append(precision)
        recalls.append(recall)
    
    results.append({'n': n, 'algorithm': 's-em',
                    'precision': np.mean(precisions), 'recall': np.mean(recalls)})
    
print(results)

[{'n': 10, 'algorithm': 's-em', 'precision': 0.51040723981900449, 'recall': 0.047259929612870781}, {'n': 20, 'algorithm': 's-em', 'precision': 0.50214932126696832, 'recall': 0.092990615049438563}, {'n': 30, 'algorithm': 's-em', 'precision': 0.49396681749622928, 'recall': 0.13721300486006369}, {'n': 50, 'algorithm': 's-em', 'precision': 0.50144796380090495, 'recall': 0.23215183509301157}, {'n': 100, 'algorithm': 's-em', 'precision': 0.48821266968325788, 'recall': 0.45204876822523876}, {'n': 150, 'algorithm': 's-em', 'precision': 0.44318250377073903, 'recall': 0.6155312552371377}, {'n': 200, 'algorithm': 's-em', 'precision': 0.39433257918552039, 'recall': 0.73024551701022278}]
CPU times: user 2.18 s, sys: 24 ms, total: 2.2 s
Wall time: 2.2 s
